In [25]:
import secrets
import string
import pandas as pd
import hashlib
import base64
import os

from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes



In [26]:
csv_path = "../modified_data/hotel_booking_token.csv"
csv_output_path = "../modified_data/hotel_booking_token_rsaemail.csv"
public_path = "public_key.pem"

In [27]:

def generate_keys():
    if os.path.exists(public_path):
      return
    # Generate private key
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    
    # Generate public key
    public_key = private_key.public_key()
    
    # Save the private key
    with open("private_key.pem", "wb") as f:
        f.write(private_key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.PKCS8,
            encryption_algorithm=serialization.NoEncryption()
        ))
    
    # Save the public key
    with open("public_key.pem", "wb") as f:
        f.write(public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        ))
# Usage
generate_keys()


In [28]:

# Load public key
with open("public_key.pem", "rb") as f:
    public_key = serialization.load_pem_public_key(
        f.read(),
        backend=default_backend()
    )
# Load private key
with open("private_key.pem", "rb") as f:
    private_key = serialization.load_pem_private_key(
        f.read(),
        password=None,
        backend=default_backend()
    )

def encrypt_message(message):
    # Encrypt message
    encrypted = base64.b64encode(public_key.encrypt(
        message.encode(),
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )).decode('utf-8')
    return encrypted

def decrypt_message(encrypted_message):
    # Decrypt message
    original_message = private_key.decrypt(
        base64.b64decode(encrypted_message),
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return original_message.decode()

encrypted_message = encrypt_message("Hello, this is a secret message!")
print("Encrypted:", encrypted_message)
decrypted_message = decrypt_message(encrypted_message)
print("Decrypted:", decrypted_message)

Encrypted: cLxxQe8ebjSTOCnJhz9Am5I/AC9UqePj9yVE7SX9q26SDxVNrxk4p5iYKC4BKsW3OOACzSqzJTTXz/wqmlpDeEYPRlHDqwRaK9ezRAwe47cpQFCwcz0jyhRWGcasOh5lTbRAjYT2HGrigkiCYaeaz55aXYh/KIuDGaYKhxsE3DwzA7Nuxv0Na2KPHqe5GsXXgFCOKEzoPSexChSRs79fmoDssJKrSnIVH8urzQh+E+n5em/eCmSxOsAU9RzDx61tt8fzNE62i9zbjY0gE53aDRi7vPfOSzb9EdYe7uje0kvqp/SsNxNHMV7aJzCveZUmOgoyjw1+y/1gXC+0OqtA9g==
Decrypted: Hello, this is a secret message!


In [29]:
total = 0
file_path = csv_path
simple_data = pd.read_csv(file_path)

total=len(simple_data)


In [30]:
encrpyted_emails = []

simple_data['email'] = simple_data['email'].apply(lambda x: encrypt_message(x))
print(simple_data['email'].values[0])
print(decrypt_message(simple_data['email'].values[0]))

simple_data.to_csv(csv_output_path, index=False)

jY3RfrGZR/hcgfEYjqCd0fLcgVxGsvL3jpHBhXECZDwn/oBc7aOb3sZpVNR418QmnAuei1P5FryzL2d4UdPRkwKfQ0c+jRI5wWvZc+G5QuUgO8SIZY2b5ugvSMc7hlJZXAx0hEXGAyX6uVSozcAe9dwpKX+dZ+ON3znYpKpRw+UwpULh9f+yDcsayo4EtMp3PEa63gezsv8R+6l/Ijnw0wTETT/k3fVPRHR48kEznMXlx025ZXTX2lFA3Z9+lFW7Da6FqiJnA8pJXVF++XaRF/DBbYvp/Pnfdz46uuftHQB2kQHfqtPvDe6RKKbTG40K9wzR7t1MTE8xt6vgc6zANw==
Ernest.Barnes31@outlook.com
